## Using google maps API to evaluate circuity factor
by Guilherme Fernandes Alves, May 2022

### Load required libraries

In [1]:
import pandas as pd
import googlemaps
from math import radians, cos, sin, asin, sqrt

### Setting up the Google Maps API

In [3]:
# Load googlemaps by using the API key
# API keys are generated in the 'Credentials' page, refer to following:
# https://developers.google.com/maps/documentation/geocoding/get-api-key
gmaps = googlemaps.Client(key="Add Your Key here")

### Load both origins and destinations data available on excel file 

In [ ]:
wb_origins = pd.read_csv("origins.csv", encoding="utf-8")

wb_destinations = pd.read_csv("destinations.csv", encoding="utf-8")

Visualize origin points

In [ ]:
wb_origins.head()

Visualize destination points

In [ ]:
wb_destinations.head()

In [ ]:
def print_matrix_length(wb_origins, wb_destinations):
    """Simple function to print the length of the matrix

    Parameters
    ----------
    wb_origins : pandas.DataFrame
        Origin points
    wb_destinations : pandas.DataFrame
        Destination points
    """
    print(
        "The matrix will have {} rows and {} columns".format(
            len(wb_origins), len(wb_destinations)
        )
    )
    print(
        "Number of combinations Origin x Destinations: ",
        len(wb_destinations) * len(wb_origins),
    )


print_matrix_length(wb_origins, wb_destinations)

### Define auxiliary functions

In [4]:
from lmr_analyzer.utils import Haversine as haversine

### Start running the evaluation

In [ ]:
def calculate_distanceMatrix_table(wb_origins, wb_destinations):
    distances = [
        [
            "Origin",
            "Lat0",
            "Lon0",
            "Destination",
            "Lat1",
            "Lon1",
            "Driving Distance (m)",
            "Euclidean Distance (m)",
            "Circuity Factor",
        ]
    ]

    # Iterate over the origins
    for _, l_origin in wb_origins.iterrows():
        # Iterate over the destinations
        for _, l_destination in wb_destinations.iterrows():
            # Calculate the drive distance between the origin and the destination
            getAPI = gmaps.distanceMatrix(
                str(l_origin["Lat0"]) + ", " + str(l_origin["Lon0"]),
                str(l_destination["Lat1"]) + ", " + str(l_destination["Lon1"]),
            )

            # Save the drive distance in a temporary variable
            d_maps = getAPI["rows"][0]["elements"][0]["distance"]["value"]

            # Calculate the euclidean distance between the origin and the destination in meters
            euclidean = 1000 * haversine(
                lat1=l_origin["Lat0"],
                lon1=l_origin["Lon0"],
                lat2=l_destination["Lat1"],
                lon2=l_destination["Lon1"],
            )

            # Calculate the circuity factor
            try:
                circuity_factor = d_maps / euclidean
            except ZeroDivisionError:
                circuity_factor = None

            # Append the results to the distances list
            distances.append(
                [
                    str(l_origin["Origin"]),
                    l_origin["Lat0"],
                    l_origin["Lon0"],
                    l_destination["Destination"],
                    l_destination["Lat1"],
                    l_destination["Lon1"],
                    d_maps,
                    euclidean,
                    circuity_factor,
                ]
            )

    return distances


# Run the function
distances = calculate_distanceMatrix_table(wb_origins, wb_destinations)
distances

Write Excel file whitin distance matrix combinations

In [ ]:
with open("distances.csv", "w", encoding="utf-8") as f:
    for item in distances:
        f.write("%s \n" % item)